In [27]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from datetime import timedelta

import squarify

In [29]:
df=pd.read_csv(r"C:\Users\hp\Downloads\Retail_Data_Transactions.csv")

In [31]:
df.head()

,customer_id,trans_date,tran_amount
0,CS5295,11-Feb-17,35
1,CS4768,15-Mar-19,39
2,CS2122,26-Feb-17,52
3,CS1217,16-Nov-15,99
4,CS1850,20-Nov-17,78


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  125000 non-null  object
 1   trans_date   125000 non-null  object
 2   tran_amount  125000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.9+ MB


In [41]:
df['trans_date']=pd.to_datetime(df['trans_date'])

In [106]:
df.head()

,customer_id,trans_date,tran_amount
0,CS5295,2017-02-11,35
1,CS4768,2019-03-15,39
2,CS2122,2017-02-26,52
3,CS1217,2015-11-16,99
4,CS1850,2017-11-20,78


### Identifying Unique Customers

In [49]:
df['customer_id'].nunique()

6889

In [66]:
print("Transaction timeframe from {} to {}".format(df['trans_date'].min(),df['trans_date'].max()))

Transaction timeframe from 2015-05-16 00:00:00 to 2019-03-16 00:00:00


In [70]:
print("shape of df row {:,} and columns {:,}".format(df.shape[0],df.shape[1]))

shape of df row 125,000 and columns 3


In [82]:
print("Tranactions with no customerid {:,}".format(df[df['customer_id'].isnull()].shape[0]))

Tranactions with no customerid 0


In [76]:
df.shape

(125000, 3)

In [102]:
## create snapshot date
snapshot_date=df.trans_date.max()+timedelta(days=1)
snapshot_date

Timestamp('2019-03-17 00:00:00')

In [194]:
### creating groupby customerid

Dataprocess=df.groupby(['customer_id']).agg({
    'trans_date':lambda x:(snapshot_date-x.max()).days,
    'customer_id':'count',
    'tran_amount':'sum'})

In [196]:
Dataprocess.head()

,trans_date,customer_id,tran_amount
customer_id,,,
CS1112,62,15,1012
CS1113,36,20,1490
CS1114,33,19,1432
CS1115,12,22,1659
CS1116,204,13,857


In [198]:
Dataprocess.shape

(6889, 3)

In [200]:
data=Dataprocess.rename(columns={'trans_date':'Recency','customer_id':'frequency','tran_amount':'monetory'})
data.head()

,Recency,frequency,monetory
customer_id,,,
CS1112,62,15,1012
CS1113,36,20,1490
CS1114,33,19,1432
CS1115,12,22,1659
CS1116,204,13,857


In [202]:
r_labels=range(4,0,-1)
f_labels=range(0,4)
m_labels=range(0,4)

In [210]:
r_groups=pd.qcut(data['Recency'],q=4,labels=r_labels)
f_groups=pd.qcut(data['frequency'],q=4,labels=f_labels)
m_groups=pd.qcut(data['monetory'],q=4,labels=m_labels)

In [212]:
data=data.assign(Recency_rank=r_groups.values,Frequency_rank=f_groups.values,Monetory_rank=m_groups.values)

In [214]:
data.head()

,Recency,frequency,monetory,Recency_rank,Frequency_rank,Monetory_rank
customer_id,,,,,,
CS1112,62,15,1012,2,1,1
CS1113,36,20,1490,3,2,2
CS1114,33,19,1432,3,2,2
CS1115,12,22,1659,4,2,3
CS1116,204,13,857,1,0,1


In [224]:
data['concatrank']=data['Recency_rank'].astype(str)+data['Frequency_rank'].astype(str)+data['Monetory_rank'].astype(str)
data.head()

,Recency,frequency,monetory,Recency_rank,Frequency_rank,Monetory_rank,concatrank
customer_id,,,,,,,
CS1112,62,15,1012,2,1,1,211
CS1113,36,20,1490,3,2,2,322
CS1114,33,19,1432,3,2,2,322
CS1115,12,22,1659,4,2,3,423
CS1116,204,13,857,1,0,1,101


In [226]:
data['concatrank'].nunique()

48

In [242]:
uniquecombinations=data.groupby(['concatrank'])['concatrank'].nunique()
uniquecombinations.sum()

np.int64(48)

In [298]:
data['rfmscore']=data[['Recency_rank','Frequency_rank','Monetory_rank']].sum(axis=1)
data.head()

,Recency,frequency,monetory,Recency_rank,Frequency_rank,Monetory_rank,concatrank,rfmscore,customer_segmentation
customer_id,,,,,,,,,
CS1112,62,15,1012,2,1,1,211,4,needs attention
CS1113,36,20,1490,3,2,2,322,7,loyal
CS1114,33,19,1432,3,2,2,322,7,loyal
CS1115,12,22,1659,4,2,3,423,9,loyal
CS1116,204,13,857,1,0,1,101,2,Require Activation


In [254]:
data['rfmscore'].value_counts()

rfmscore
5     851
6     844
4     824
8     806
7     755
3     737
1     550
2     548
9     536
10    438
Name: count, dtype: int64

In [278]:
def rfmlevel (df):
    if df['rfmscore']>=7:
        return 'loyal'
    elif ((df['rfmscore']>=5)and (df['rfmscore']<7)):
        return 'champoins'
    elif ((df['rfmscore']>=4)and (df['rfmscore']<5)):
        return 'needs attention'
    else:
        return 'Require Activation'

data['customer_segmentation']=data.apply(rfmlevel,axis=1)
    

In [284]:
data.customer_segmentation.value_counts()

customer_segmentation
loyal                 2535
Require Activation    1835
champoins             1695
needs attention        824
Name: count, dtype: int64

### submission

In [288]:
report=data[['customer_segmentation']].reset_index()
report

,customer_id,customer_segmentation
0,CS1112,needs attention
1,CS1113,loyal
2,CS1114,loyal
3,CS1115,loyal
4,CS1116,Require Activation
...,...,...
6884,CS8996,Require Activation
6885,CS8997,Require Activation
6886,CS8998,Require Activation
6887,CS8999,Require Activation


In [300]:
Pareto=pd.pivot_table(data,index=["customer_segmentation"],values=["monetory"],aggfunc=[np.sum,len])
Pareto

,sum,len
,monetory,monetory
customer_segmentation,,
Require Activation,1235185,1835
champoins,2033696,1695
loyal,4119692,2535
needs attention,735416,824


In [312]:
rfm_level_agg = data.groupby('customer_segmentation').agg({
    'Recency': 'mean',
    'frequency': 'mean',
    'monetory': ['mean', 'count']
}).round()

rfm_level_agg.columns = ['RecencyMean','FrequencyMean','MonetaryMean', 'Count']
rfm_level_agg

,RecencyMean,FrequencyMean,MonetaryMean,Count
customer_segmentation,,,,
Require Activation,154.0,13.0,673.0,1835
champoins,69.0,18.0,1200.0,1695
loyal,41.0,23.0,1625.0,2535
needs attention,70.0,15.0,892.0,824
